In [2]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO, Entrez
from io import StringIO

In [18]:
# parse the blastp output file

sequences = []

# Parse the results from the files
with open("./proteins/keratin/keratin-blastp.xml") as in_handle:
    blast_records = NCBIXML.parse(in_handle)
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < 0.0001:
                    print("****Alignment****")
                    print("sequence:", alignment.title)
                    print("length:", alignment.length)
                    print("e value:", hsp.expect)
                    print(hsp.query[0:75] + "...")
                    print(hsp.match[0:75] + "...")
                    print(hsp.sbjct[0:75] + "...")
        

****Alignment****
sequence: gb|EAW58243.1| hCG1997648 [Homo sapiens] >gb|KAI4066085.1| keratin 81 [Homo sapiens] >dbj|BAI45776.1| keratin 81, partial [synthetic construct] >emb|CAA73943.1| keratin [Homo sapiens]
length: 505
e value: 0.0
MTCGSGFGGRAFSCISACGPRPGRCCITAAPYRGISCYRGLTGGFGSHSVCGGFRAGSCGRSFGYRSGGVCGPSP...
MTCGSGFGGRAFSCISACGPRPGRCCITAAPYRGISCYRGLTGGFGSHSVCGGFRAGSCGRSFGYRSGGVCGPSP...
MTCGSGFGGRAFSCISACGPRPGRCCITAAPYRGISCYRGLTGGFGSHSVCGGFRAGSCGRSFGYRSGGVCGPSP...
****Alignment****
sequence: ref|XP_018893534.2| keratin, type II cuticular Hb1 [Gorilla gorilla gorilla]
length: 505
e value: 0.0
MTCGSGFGGRAFSCISACGPRPGRCCITAAPYRGISCYRGLTGGFGSHSVCGGFRAGSCGRSFGYRSGGVCGPSP...
MTCGSGFGGRAFSCISACGPRPGRCCITAAPYRGISCYRGLTGGFGSHSVCGGFRAGSCGRSFGYRSGGVCGPSP...
MTCGSGFGGRAFSCISACGPRPGRCCITAAPYRGISCYRGLTGGFGSHSVCGGFRAGSCGRSFGYRSGGVCGPSP...
****Alignment****
sequence: ref|XP_024112725.1| keratin, type II cuticular Hb1 [Pongo abelii] >gb|PNJ25503.1| KRT86 isoform 1 [Pongo abelii]
length: 505
e valu

# Byte Pair Encoding (BPE)


Introduced by the paper [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/abs/1508.07909). Consists in bring small pieces of the sequence and tokenize them reling on the sequence frequency. The intuition behind this approach encode the contex of each sub-component, avoiding the issue of out-of-word vocabulary.

In [8]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:0000:0100:01


In [75]:
import sentencepiece as spm
import glob
import os
import pickle

def from_xml_to_BPE_protein_tokenization():
    """Encode the sequences using BPE.

    Args:
        None

    Returns:    
        None
    
    """

    # Open the BLAST XML output file
    blast_xml_files = glob.glob("./proteins/*/*-blastp.xml")

    # vocab size
    # vocab_s = dict([(file.split('/')[2],max([len(seq) for seq in open(file, 'r').readlines()])*15) for file in glob.glob("./proteins/*/sequences/sequences.txt")])
    vocab_s = dict([(file.split('/')[2],max([len(seq) for seq in open(file, 'r').readlines()])*15) for file in glob.glob("./proteins/*/sequences/sequences.txt")])

    vocab_s['insulin'] = 6000
    vocab_s['hemoglobin'] = 6722
    vocab_s['erythropoietin'] = 8000
    vocab_s['collagen'] = int(vocab_s['collagen']/3)
    vocab_s['myosin'] = int(vocab_s['myosin']/3)
    vocab_s['trypsin'] = int(vocab_s['trypsin']/2)
    vocab_s['elastin'] = 8000
    vocab_s['tubulin'] = 3421

    for blast_xml_file in blast_xml_files:

        # Extract the protein name from the file path
        prot = blast_xml_file.split("/")[2]

        # https://github.com/google/sentencepiece/blob/master/doc/options.md
        spm.SentencePieceTrainer.train(
            input=f'./proteins/{prot}/sequences/sequences.txt', 
            model_type='bpe', 
            shuffle_input_sentence=False,
            split_by_whitespace=False,
            max_sentencepiece_length=16,
            allow_whitespace_only_pieces=True,
            model_prefix=f'BPE_model_{prot}', 
            vocab_size=vocab_s[prot]
        )

        sm = spm.SentencePieceProcessor()
        # load the model
        sm.load(f'BPE_model_{prot}.model')

        # Parse the BLAST output file
        with open(blast_xml_file) as blast_xml:
            blast_record = NCBIXML.read(blast_xml)

        # Extract the sequences from the BLAST output file and write to FASTA file
        sequences = []
        sequences_str = []

        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:

                # Format the sbjct sequence as a FASTA record
                accession = alignment.hit_id.split("|")[1]
                sbjct_record = ">{} {}_{}\n{}\n".format(accession,alignment.hit_def, hsp.sbjct_start, hsp.sbjct)

                # Parse the FASTA record using SeqIO.read()
                seq_record = SeqIO.read(StringIO(sbjct_record), "fasta")
                
                # encode the sequence
                sequences_str.append((accession,sm.encode(str(seq_record.seq), out_type=str)))
                sequences.append((accession,sm.encode(str(seq_record.seq))))

        # Write the encoded sequences into a pickle file
        with open(f"./proteins/{prot}/sequences/sequences_BPE.pkl", "wb") as f:
            pickle.dump(sequences, f)
        with open(f"./proteins/{prot}/sequences/sequences_BPE_str.pkl", "wb") as f:
            pickle.dump(sequences_str, f)

        # move the model+vocab to the right folder
        os.rename(f"./BPE_model_{prot}.model", f"./proteins/{prot}/sequences/BPE_model_{prot}.model")
        os.rename(f"./BPE_model_{prot}.vocab", f"./proteins/{prot}/sequences/BPE_model_{prot}.vocab")



from_xml_to_BPE_protein_tokenization()


In [64]:
# dictionary with the dimension of the vocabulary for each protein
d = dict([(file.split('/')[2],max([len(seq) for seq in open(file, 'r').readlines()])*15) for file in glob.glob("./proteins/*/sequences/sequences.txt")])

d['insulin'] = 6000
d['hemoglobin'] = 6792
d['erythropoietin'] = 8000
d['collagen'] = int(d['collagen']/3)
d['myosin'] = int(d['myosin']/3)
d['trypsin'] = int(d['trypsin']/2)
d['elastin'] = 8000
d['tubulin'] = 3421

d

{'aldehyde': 8400,
 'protein_kinase': 8040,
 'collagen': 7780,
 'fibrinogen': 7035,
 'albumin': 9300,
 'insulin': 6000,
 'hemoglobin': 6792,
 'erythropoietin': 8000,
 'myosin': 9835,
 'immunoglobulin': 2145,
 'tubulin': 3421,
 'keratin': 8025,
 'catalase': 8565,
 'trypsin': 9142,
 'elastin': 8000}

In [68]:
with open('./proteins/insulin/sequences/sequences_BPE.pkl', 'rb') as f:
    insulin = pickle.load(f)

insulin = dict(insulin)

insulin

{'QMS45324.1': [106, 164, 264, 49, 195, 171, 157, 50, 82],
 'AAP36446.1': [106, 164, 264, 49, 195, 171, 157, 50, 82],
 'NP_000198.1': [106, 164, 264, 49, 195, 171, 157, 50, 82],
 'XP_024110665.1': [106, 164, 153, 49, 195, 171, 157, 50, 82],
 'AKI70564.1': [106, 164, 264, 2294, 195, 171, 157, 50, 82],
 'QMS45321.1': [106, 164, 264, 49, 195, 171, 157, 50, 82],
 'NP_001008996.1': [211, 1189, 698, 49, 195, 171, 157, 50, 82],
 'AKI70566.1': [106, 164, 264, 49, 195, 171, 1847, 167, 82],
 'XP_034787832.1': [106, 164, 698, 49, 195, 171, 157, 50, 82],
 'AKI70567.1': [106, 164, 264, 49, 195, 171, 157, 1887, 82],
 'XP_050613945.1': [106, 164, 249, 49, 213, 171, 157, 50, 82],
 'AKI70565.1': [106, 164, 264, 49, 2113, 171, 157, 50, 82],
 'XP_016775240.1': [211, 1189, 698, 49, 195, 171, 157, 50, 82],
 'XP_003281399.1': [106, 164, 249, 49, 213, 171, 157, 50, 82],
 'XP_032009711.1': [656, 164, 249, 49, 213, 171, 157, 50, 82],
 'XP_003909425.2': [106, 164, 380, 49, 213, 171, 157, 50, 82],
 'XP_008002825

In [71]:
with open('./proteins/hemoglobin/sequences/sequences_BPE.pkl', 'rb') as f:
    hemoglobin = pickle.load(f)

hemoglobin = dict(hemoglobin)

for idx in hemoglobin.keys():
    print(len(hemoglobin[idx]))

13
13
13
13
12
13
13
13
13
12
13
13
13
12
13
13
12
13
13
13
13
13
13
13
13
13
13
13
13
13
14
12
13
12
13
13
13
13
12
13
12
13
13
13
12
13
12
13
14
13
13
13
13
13
13
13
13
13
12
14
13
13
13
13
13
12
13
13
12
13
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
12
13
13
13
13
13
13
13
13
12
12
12
12
13
13
13
13
13
13
14
13
12
13
12
12
13
13
13
12
12
13
12
15
13
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
24
13
12
12
13
13
13
13
13
13
13
13
11
13
13
13
13
13
13
13
13
12
13
12
13
12
12
13
13
13
13
13
13
13
12
12
13
13
13
12
13
13
12
10
13
13
12
12
12
11
11
12
12
11
13
13
13
12
12
11
12
12
12
12
12
12
12
12
12
12
13
12
12
12
12
12
12
13
12
11
12
12
13
12
13
12
12
11
12
10
12
12
12
12
12
12
12
11
13
13
12
12
12
12
12
12
13
13
12
12
12
13
12
13
13
13
12
12
12
12
12
12
12
12
13
13
13
12
12
12
13
12
12
13
12
13
12
12
13
13
12
12
12
13
13
12
12
12
12
13
13
12
13
13
12
12
13
12
12
12
12
12
13
13
13
12
13
12
12
13
12
13
13
13
13
12
13
12
13
13
12
12
13
13
13
12
1

In [77]:
files = glob.glob("./proteins/*/sequences/sequences.txt")

# load the sequences in a single file
for file in files:
    with open(file, 'rb') as f:
        with open('./sequences.txt', 'ab') as f2:
            f2.write(f.read())
            f2.write(b'\n')

In [165]:
spm.SentencePieceTrainer.train(
            input=f'./sequences.txt', 
            model_type='bpe', 
            shuffle_input_sentence=False,
            split_by_whitespace=False,
            max_sentencepiece_length=16,
            allow_whitespace_only_pieces=True,
            model_prefix=f'BPE_model', 
            vocab_size=3000
        )

In [171]:
files = glob.glob("./proteins/*/sequences/sequences_pck.pkl")

import numpy as np

# load the sequence
with open(files[0], 'rb') as f:
    seq = pickle.load(f)


sm = spm.SentencePieceProcessor()
# load the model
sm.load(f'BPE_model.model')

# list with all the encoded sequences
encoded_seq = [np.array(sm.encode(str(seq[key]))) for key in seq.keys()]

# extract maximum dimension
max_dim = np.array([len(seq) for seq in encoded_seq]).max()

# how much padding is needed
padding = [max_dim-a.shape[0] for a in encoded_seq]

# pad the sequences + 1 to avoid to use -1 as padding
encoded_seq = [np.pad(a,(0,p), constant_values=(-1, -1)) for a, p in zip(encoded_seq,padding)]

In [185]:
zero_like = sum((encoded_seq[0]-encoded_seq[1]) == 0) 

zero_like

182